In [1]:
import pyomo.environ as pyo
import pyomo.dae as dae
import numpy as np

# Simple Model
$$\frac{dT}{dt}  C = \dot{Q}_{htg} + \dot{Q}_{sol}$$


## Implementierung manuell "linearisiert"

In [2]:
# Manuelle Implementierung Simples Modell

model = pyo.ConcreteModel()

model.t = pyo.RangeSet(0, 5)
model.p = pyo.RangeSet(0, 4)

model.T = pyo.Var(model.t, bounds=(18, 22))
model.Q_htg = pyo.Var(model.p, domain=pyo.Reals)

Q_sol = [20, 20, 20, 20, 20] # lambda x: np.interp(x, range(6), [20, 20, 20, 20, 20, 20])
C = 1

@model.Constraint()
def _initial_condition(m):
   return m.T[0] ==  20

@model.Constraint(model.p)
def _ode_rule(m, p):
   return m.T[p+1] * C == m.T[p] *C + m.Q_htg[p] + Q_sol[p] 

@model.Objective()
def _objective_fun(m):
   return sum([m.Q_htg[p] for p in m.p])

# model.pprint()

solver = pyo.SolverFactory('appsi_highs')
result = solver.solve(model)

np.array(pyo.value(model.Q_htg[:]))

array([-22., -20., -20., -20., -20.])

## Implementierung mit DAE und Backward Euler

In [3]:
# Backward Euler
import pyomo.environ as pyo
import pyomo.dae as dae

import numpy as np

model = pyo.ConcreteModel()

model.t = dae.ContinuousSet(initialize=[0, 1, 2, 3, 4, 5])
model.p = dae.ContinuousSet(initialize=[1, 2, 3, 4, 5])

model.T = pyo.Var(model.t, bounds=(18, 22))
model.dTdt = dae.DerivativeVar(model.T, wrt=model.t)
model.Q_htg = pyo.Var(model.p, domain=pyo.Reals)

Q_sol = [20, 20, 20, 20, 20, 20] # lambda x: np.interp(x, range(6), [20, 20, 20, 20, 20, 20])
C = 1

@model.Constraint()
def _initial_condition(m):
   return m.T[0] ==  20

@model.Constraint(model.p)
def _ode_rule(m, p):
   return m.dTdt[p] * C ==  m.Q_htg[p] + Q_sol[p]

@model.Objective()
def _objective_fun(m):
   return sum([m.Q_htg[p] for p in m.p])

discretizer = pyo.TransformationFactory('dae.finite_difference')
discretizer.apply_to(model, wrt=model.t, nfe=5, scheme='BACKWARD') # scheme='FORWARD'

# model.pprint()

solver = pyo.SolverFactory('appsi_highs')
result = solver.solve(model)

np.array(pyo.value(model.Q_htg[:]))

array([-22., -20., -20., -20., -20.])

## Implementierung mit DAE und Forward Euler

In [4]:
# Forward Euler
import pyomo.environ as pyo
import pyomo.dae as dae

import numpy as np

model = pyo.ConcreteModel()

model.t = dae.ContinuousSet(initialize=[0, 1, 2, 3, 4, 5])
model.p = dae.ContinuousSet(initialize=[0, 1, 2, 3, 4])

model.T = pyo.Var(model.t, bounds=(18, 22))
model.dTdt = dae.DerivativeVar(model.T, wrt=model.t)
model.Q_htg = pyo.Var(model.p, domain=pyo.Reals)

Q_sol = [20, 20, 20, 20, 20, 20] # lambda x: np.interp(x, range(6), [20, 20, 20, 20, 20, 20])
C = 1

@model.Constraint()
def _initial_condition(m):
   return m.T[0] ==  20

@model.Constraint(model.p)
def _ode_rule(m, p):
   return m.dTdt[p] * C ==  m.Q_htg[p] + Q_sol[p]

@model.Objective()
def _objective_fun(m):
   return sum([m.Q_htg[p] for p in m.p])

discretizer = pyo.TransformationFactory('dae.finite_difference')
discretizer.apply_to(model, wrt=model.t, nfe=5, scheme='FORWARD') 

# model.pprint()

solver = pyo.SolverFactory('appsi_highs')
result = solver.solve(model)

np.array(pyo.value(model.Q_htg[:]))

array([-22., -20., -20., -20., -20.])

# Slightly more complex Model
$$\frac{dT}{dt} C = \dot{Q}_{htg} + \dot{Q}_{sol} - UA * (T - T_{\infty})$$

In [5]:
# Forward Euler
import pyomo.environ as pyo
import pyomo.dae as dae

import numpy as np

model = pyo.ConcreteModel()

model.t = dae.ContinuousSet(initialize=[0, 1, 2, 3, 4, 5])
model.p = dae.ContinuousSet(initialize=[0, 1, 2, 3, 4])

model.T = pyo.Var(model.t, bounds=(18, 22))
model.dTdt = dae.DerivativeVar(model.T, wrt=model.t)
model.Q_htg = pyo.Var(model.p, domain=pyo.Reals)

Q_sol = [20, 20, 20, 20, 20] # lambda x: np.interp(x, range(6), [20, 20, 20, 20, 20, 20])
T_inf = [10, 10, 20, 20, 10] # lambda x: np.interp(x, range(6), [20, 20, 20, 20, 20, 20])
C = 1
UA = 2

@model.Constraint()
def _initial_condition(m):
   return m.T[0] ==  20

@model.Constraint(model.p)
def _ode_rule(m, p):
   return m.dTdt[p] * C ==  m.Q_htg[p] + Q_sol[p] - UA * (m.T[p] - T_inf[p])

@model.Objective()
def _objective_fun(m):
   return sum([m.Q_htg[p] for p in m.p])

discretizer = pyo.TransformationFactory('dae.finite_difference')
discretizer.apply_to(model, wrt=model.t, nfe=5, scheme='FORWARD') 

# model.pprint()

solver = pyo.SolverFactory('appsi_highs')
result = solver.solve(model)

np.array(pyo.value(model.Q_htg[:]))

array([ -2.,  -4., -24., -24.,  -4.])

# Coupled Differential Equations Model
$$\frac{dT_1}{dt} C_1 = \dot{Q}_{htg} + \dot{Q}_{sol} - UA_{\infty} (T_1 - T_{\infty}) - UA_{1,2} (T_1 - T_2)$$
$$\frac{dT_2}{dt} C_2 = UA_{1,2} (T_1 - T_2)$$

In [6]:
# Forward Euler
import pyomo.environ as pyo
import pyomo.dae as dae

import numpy as np

model = pyo.ConcreteModel()

model.t = dae.ContinuousSet(initialize=[0, 1, 2, 3, 4, 5])
model.p = dae.ContinuousSet(initialize=[0, 1, 2, 3, 4])

model.T_1 = pyo.Var(model.t, bounds=(18, 22))
model.dT_1dt = dae.DerivativeVar(model.T_1, wrt=model.t)
model.T_2 = pyo.Var(model.t, bounds=(16, 24))
model.dT_2dt = dae.DerivativeVar(model.T_2, wrt=model.t)
model.Q_htg = pyo.Var(model.p, domain=pyo.Reals)

Q_sol = [20, 20, 20, 20, 20] # lambda x: np.interp(x, range(6), [20, 20, 20, 20, 20, 20])
T_inf = [10, 10, 20, 20, 10] # lambda x: np.interp(x, range(6), [20, 20, 20, 20, 20, 20])
C_1 = 1
C_2 = 1
UA_inf = 2
UA_1_2 = 2

@model.Constraint()
def _initial_condition_1(m):
   return m.T_1[0] ==  20

@model.Constraint()
def _initial_condition_2(m):
   return m.T_2[0] ==  20

@model.Constraint(model.p)
def _ode_rule_1(m, p):
   return m.dT_1dt[p] * C_1 ==  m.Q_htg[p] + Q_sol[p] - UA_inf * (m.T_1[p] - T_inf[p]) - UA_1_2 * (m.T_1[p] - m.T_2[p])

@model.Constraint(model.p)
def _ode_rule_2(m, p):
   return m.dT_2dt[p] * C_2 ==  UA_1_2 * (m.T_1[p] - m.T_2[p])


@model.Objective()
def _objective_fun(m):
   return sum([m.Q_htg[p] for p in m.p])

discretizer = pyo.TransformationFactory('dae.finite_difference')
discretizer.apply_to(model, wrt=model.t, nfe=5, scheme='FORWARD') 

# model.pprint()

solver = pyo.SolverFactory('appsi_highs')
result = solver.solve(model)

np.array(pyo.value(model.Q_htg[:]))

array([ -2.,  -8., -18., -22.,  -8.])

# Transformation Factory umschreiben mit Matrixexponential
- Alle Nebenbedingungen und Zielfunktionen mit DerivativeVars finden
- Sortieren nach Derivative Vars, State Vars und Konstanten
- A-Matrix und B-Vektor erstellen
- System diskretisieren mittels Matrixexponential (https://en.wikipedia.org/wiki/Discretization)
- Matrix in Nebenbedingungen umwandeln und dem Modell hinzufügen
- Alte Nebenbedingungen deaktivieren (sodass sie bei einer Rücktransformation wieder aktiviert werden könnten)

In [ ]:
# Transformation factory der Finite Difference Methode (als Vorlage):

@TransformationFactory.register(
    'dae.finite_difference',
    doc="Discretizes a DAE model using "
    "a finite difference method transforming the model into an NLP.",
)
class Finite_Difference_Transformation(Transformation):
    """
    Transformation that applies finite difference methods to
    DAE, ODE, or PDE models.
    """

    CONFIG = ConfigBlock("dae.finite_difference")
    CONFIG.declare(
        'nfe',
        ConfigValue(
            default=10,
            domain=PositiveInt,
            description="The desired number of finite element points to be "
            "included in the discretization",
        ),
    )
    CONFIG.declare(
        'wrt',
        ConfigValue(
            default=None,
            description="The ContinuousSet to be discretized",
            doc="Indicates which ContinuousSet the transformation should be "
            "applied to. If this keyword argument is not specified then the "
            "same scheme will be applied to all ContinuousSets.",
        ),
    )
    CONFIG.declare(
        'scheme',
        ConfigValue(
            default='BACKWARD',
            domain=In(['BACKWARD', 'CENTRAL', 'FORWARD']),
            description="Indicates which finite difference scheme to apply",
            doc="Options are BACKWARD, CENTRAL, or FORWARD. The default scheme is "
            "the backward difference method",
        ),
    )

    def __init__(self):
        super(Finite_Difference_Transformation, self).__init__()
        self._nfe = {}
        self.all_schemes = {
            'BACKWARD': (_backward_transform, _backward_transform_order2),
            'CENTRAL': (_central_transform, _central_transform_order2),
            'FORWARD': (_forward_transform, _forward_transform_order2),
        }

    def _apply_to(self, instance, **kwds):
        """
        Applies the transformation to a modeling instance

        Keyword Arguments:
        nfe           The desired number of finite element points to be
                      included in the discretization.
        wrt           Indicates which ContinuousSet the transformation
                      should be applied to. If this keyword argument is not
                      specified then the same scheme will be applied to all
                      ContinuousSets.
        scheme        Indicates which finite difference method to apply.
                      Options are BACKWARD, CENTRAL, or FORWARD. The default
                      scheme is the backward difference method
        """

        config = self.CONFIG(kwds)

        tmpnfe = config.nfe
        tmpds = config.wrt

        if tmpds is not None:
            if tmpds.ctype is not ContinuousSet:
                raise TypeError(
                    "The component specified using the 'wrt' "
                    "keyword must be a continuous set"
                )
            elif 'scheme' in tmpds.get_discretization_info():
                raise ValueError(
                    "The discretization scheme '%s' has already "
                    "been applied to the ContinuousSet '%s'"
                    % (tmpds.get_discretization_info()['scheme'], tmpds.name)
                )

        if None in self._nfe:
            raise ValueError(
                "A general discretization scheme has already been applied to "
                "to every continuous set in the model. If you would like to "
                "apply a different discretization scheme to each continuous "
                "set, you must declare a new transformation object"
            )

        if len(self._nfe) == 0 and tmpds is None:
            # Same discretization on all ContinuousSets
            self._nfe[None] = tmpnfe
            currentds = None
        else:
            self._nfe[tmpds.name] = tmpnfe
            currentds = tmpds.name

        self._scheme_name = config.scheme
        self._scheme = self.all_schemes.get(self._scheme_name, None)

        self._transformBlock(instance, currentds)

    def _transformBlock(self, block, currentds):
        self._fe = {}
        for ds in block.component_objects(ContinuousSet):
            if currentds is None or currentds == ds.name or currentds is ds:
                if 'scheme' in ds.get_discretization_info():
                    raise DAE_Error(
                        "Attempting to discretize ContinuousSet "
                        "'%s' after it has already been discretized. " % ds.name
                    )
                generate_finite_elements(ds, self._nfe[currentds])
                if not ds.get_changed():
                    if len(ds) - 1 > self._nfe[currentds]:
                        logger.warning(
                            "More finite elements were found in "
                            "ContinuousSet '%s' than the number of "
                            "finite elements specified in apply. The "
                            "larger number of finite elements will be "
                            "used." % ds.name
                        )

                self._nfe[ds.name] = len(ds) - 1
                self._fe[ds.name] = list(ds)
                # Adding discretization information to the ContinuousSet
                # object itself so that it can be accessed outside of the
                # discretization object
                disc_info = ds.get_discretization_info()
                disc_info['nfe'] = self._nfe[ds.name]
                disc_info['scheme'] = self._scheme_name + ' Difference'

        # Maybe check to see if any of the ContinuousSets have been changed,
        # if they haven't then the model components need not be updated
        # or even iterated through
        expand_components(block)

        for d in block.component_objects(DerivativeVar, descend_into=True):
            dsets = d.get_continuousset_list()
            for i in ComponentSet(dsets):
                if currentds is None or i.name == currentds:
                    oldexpr = d.get_derivative_expression()
                    loc = d.get_state_var()._contset[i]
                    count = dsets.count(i)
                    if count >= 3:
                        raise DAE_Error(
                            "Error discretizing '%s' with respect to '%s'. "
                            "Current implementation only allows for taking the"
                            " first or second derivative with respect to "
                            "a particular ContinuousSet" % (d.name, i.name)
                        )
                    scheme = self._scheme[count - 1]
                    newexpr = create_partial_expression(scheme, oldexpr, i, loc)
                    d.set_derivative_expression(newexpr)

            # Reclassify DerivativeVar if all indexing ContinuousSets have
            # been discretized. Add discretization equations to the
            # DerivativeVar's parent block.
            if d.is_fully_discretized():
                add_discretization_equations(d.parent_block(), d)
                d.parent_block().reclassify_component_type(d, Var)

                # Keep track of any reclassified DerivativeVar components so
                # that the Simulator can easily identify them if the model
                # is simulated after discretization
                # TODO: Update the discretization transformations to use
                # a Block to add things to the model and store discretization
                # information. Using a list for now because the simulator
                # does not yet support models containing active Blocks
                reclassified_list = getattr(
                    block, '_pyomo_dae_reclassified_derivativevars', None
                )
                if reclassified_list is None:
                    block._pyomo_dae_reclassified_derivativevars = list()
                    reclassified_list = block._pyomo_dae_reclassified_derivativevars

                reclassified_list.append(d)

        # Reclassify Integrals if all ContinuousSets have been discretized
        if block_fully_discretized(block):
            if block.contains_component(Integral):
                for i in block.component_objects(Integral, descend_into=True):
                    i.parent_block().reclassify_component_type(i, Expression)
                    # TODO: The following reproduces the old behavior of
                    # "reconstruct()".  We should come up with an
                    # implementation that does not rely on manipulating
                    # private attributes
                    i.clear()
                    i._constructed = False
                    i.construct()

                # If a model contains integrals they are most likely to
                # appear in the objective function which will need to be
                # reconstructed after the model is discretized.
                for k in block.component_objects(Objective, descend_into=True):
                    # TODO: check this, reconstruct might not work
                    # TODO: The following reproduces the old behavior of
                    # "reconstruct()".  We should come up with an
                    # implementation that does not rely on manipulating
                    # private attributes
                    k.clear()
                    k._constructed = False
                    k.construct()